Separando ações de interesse

In [1]:
import os
import numpy as np
import pandas as pd

input_path = os.path.join('data', 'mocked_market_stocks.csv')
df = pd.read_csv(input_path, sep=",", parse_dates=["File_Date"])
df = df[df['Company'] == "AAPL"]

# df["Datetime"] = df["Date"] + " " + df["Time"]
# df["Datetime"] = pd.to_datetime(df["Datetime"])
# df = df.drop(["Date", "Time"], axis=1)
df = df.drop(["Date"], axis=1)

# df = df[['File_Date', 'Datetime', 'Shares_Top_of_Book', 'Prices_Top_of_Book', 'Transaction_type']]
df = df[['File_Date', 'Time', 'Number_of_Shares_Sold', 'Price_of_One_Share', 'Transaction_type']]

df = df.rename({'Number_of_Shares_Sold':'Shares'}, axis=1)
df = df.rename({'Price_of_One_Share':'Prices'}, axis=1)
df = df.rename({'Transaction_type':'Type'}, axis=1)

df = df.sort_values(by="File_Date")

df.head(11)

,File_Date,Time,Shares,Prices,Type
90,2021-12-01 09:30:00,09:29:52,3,50.229802,bids
91,2021-12-01 09:30:00,09:29:55,253,49.909549,bids
92,2021-12-01 09:30:00,09:29:57,100,49.865604,bids
93,2021-12-01 09:30:00,09:29:58,4,49.463831,bids
94,2021-12-01 09:30:00,09:29:59,36,49.409480,bids
95,2021-12-01 09:30:00,09:29:49,100,50.358234,asks
96,2021-12-01 09:30:00,09:29:52,100,50.178390,asks
97,2021-12-01 09:30:00,09:29:55,820,50.722098,asks
98,2021-12-01 09:30:00,09:29:56,100,48.999970,asks
99,2021-12-01 09:30:00,09:29:57,2,49.145259,asks


In [2]:
df2 = df.groupby(['File_Date', 'Type']).agg({'Shares': list, 'Prices': list}).reset_index()
df2 = df2.pivot(index='File_Date', columns='Type').reset_index()
df2.columns = ['_'.join(col).rstrip('_') for col in df2.columns.values]

for col in df2.columns:
    temp_df = pd.DataFrame(df2[col].tolist())
    temp_df.columns = [f"{col}_{i}" for i in range(len(temp_df.columns))]
    df2 = pd.concat([df2.drop(col, axis=1), temp_df], axis=1)

df2.drop(['File_Date_0'], axis=1, inplace=True)

rules = {'Prices': 'P', 'Shares': 'S', '_bids': 'B', '_asks': 'A'}
for pre, post in rules.items():
    df2.columns = df2.columns.str.replace(pre, post)

print(df2.head(10))

   SA_0  SA_1  SA_2  SA_3  SA_4  SB_0  SB_1  SB_2  SB_3  SB_4       PA_0  \
0   100   100   820   100     2     3   253   100     4    36  50.358234   
1   100   100   100   100    66   100     7    50    40   100  49.642267   
2     7   123   100   850   100   100     5     1   100    50  50.142396   
3    55    36   100     6   100    26    50    17   100   100  51.199719   
4     8    40   100   100     6     5   100     6   100   100  50.957597   
5    15    80   100     1   100     6    12    27   100    82  50.443048   
6     1    26   100   100    50    50   100   100   100   100  51.277193   
7   260   100   100   100   100   100    90    73   100    14  51.571006   
8   100    24   100     6   100    72   100   100   100    50  51.413662   
9   100     7    29     8    38     1   100    22   100   100  52.458788   

        PA_1       PA_2       PA_3       PA_4       PB_0       PB_1  \
0  50.178390  50.722098  48.999970  49.145259  50.229802  49.909549   
1  50.453023  49.8862

In [3]:
window = 3
df_lag = df2.copy()
for i in range(1, window+1):
    temp_df = df2.shift(i)
    temp_df.columns = [f'{col}_{i}' for col in temp_df.columns]
    df_lag = pd.concat([df_lag, temp_df], axis=1)

df2 = df_lag.dropna(inplace=False)
print(df2.head(10))
print(df2.columns)


    SA_0  SA_1  SA_2  SA_3  SA_4  SB_0  SB_1  SB_2  SB_3  SB_4  ...  \
3     55    36   100     6   100    26    50    17   100   100  ...   
4      8    40   100   100     6     5   100     6   100   100  ...   
5     15    80   100     1   100     6    12    27   100    82  ...   
6      1    26   100   100    50    50   100   100   100   100  ...   
7    260   100   100   100   100   100    90    73   100    14  ...   
8    100    24   100     6   100    72   100   100   100    50  ...   
9    100     7    29     8    38     1   100    22   100   100  ...   
10    32   100   790   100   100   100     1     2    50     8  ...   
11    43    25     4    31   100    40   100    50    54    50  ...   
12     8     1   100    17    50   100    14    37    46    16  ...   

       PA_0_3     PA_1_3     PA_2_3     PA_3_3     PA_4_3     PB_0_3  \
3   50.358234  50.178390  50.722098  48.999970  49.145259  50.229802   
4   49.642267  50.453023  49.886232  49.083358  49.680085  49.692246   
5 